## konlpy로 synop 명사 추출
## CountVectorizer, cosine_similarity

1. synop내 명사만으로는 유사도 계산 부정확
2. 제목, 태그, 대분류 완료 후  다시 계산
3. 불용어 제거
4. ~~다른 방송국 전체 데이터~~
5. 유사도 계산도 트레이닝이 필요한 것인가?
6. TfidfVectorizer은 다른 nb에 작성(-> TF, IDF 계산 식에 가중치 부여 가능여부?)
7. 각 idx에 추천 epi idx column 추가 후 csv파일 저장

In [90]:
import pandas as pd
import numpy as np
import unicodedata
from konlpy.tag import *
#from ast import literal_eval

In [92]:
metadata = pd.read_csv('data_all_nsynop_dayoung_ver2.csv', encoding='cp949')
del metadata['synop']

In [93]:
metadata.head()

,idx,tv,title,epi,dates,links,vod_key,n_synop
0,0,MBC,MBC 스페셜,"생존 체력, 약골 40대 저질체력 탈출기",2018.10.01,http://www.imbc.com/broad/tv/culture/mbcspecia...,"체력', '나이', '정신', '일상', '운동', '40대', '시간', '아이'...",마흔 살 생존 체력 필요 불혹 나이 생애 전환기 신체적 정신적 삶 터닝포인트 40세...
1,1,MBC,MBC 스페셜,"당신, 독을 먹고 있나요?",2018.09.10,http://www.imbc.com/broad/tv/culture/mbcspecia...,"달콤함', '인도', '설탕', '과거', '섭취', '단맛'",스위트 허니문(Honeymoon) 고진감래(苦盡甘來)… 달콤함 우리 행복 즐거움 리...
2,2,MBC,MBC 스페셜,방송의 날 특집6개월 후 만납시다: 북한 결핵병원 이야기,2018.09.03,http://www.imbc.com/broad/tv/culture/mbcspecia...,"한반도', '북한', '이야기', '의사', '환자들', '치료', '다큐멘터리',...",2018년 한반도 급변 시작 11년 만 성사 남북정상회담 2년 10개월 만 이산가족...
3,3,MBC,MBC 스페셜,이산가족 상봉 특집 <옥류관 서울 1호점> 3부 이산,2018.08.20,http://www.imbc.com/broad/tv/culture/mbcspecia...,"이산', '이산가족', '상봉', '행사', '70년', '길자', '평양', '고...",역사적 4.27 남북정상회담 계기 8월 20일 26일 2015년 이후 중단 이산가족...
4,4,MBC,MBC 스페셜,도시x자연다큐멘터리Ⅰ 고냥이,2018.08.13,http://www.imbc.com/broad/tv/culture/mbcspecia...,"도시', '길고양이', '고양이', '영역', '어미', '불청객', '새끼들'",1 가구 증가 고령화 사회 속 반려동물 인구 천만명 반려동물 인구 위 동물들 무관심...


In [94]:
metadata['n_synop'][755]

'2부작 옥수수 습격 1부 풀 기억동물성 지방 사람들미국 사우스캐롤라이나 지미 무어씨 하루 300g 버터 하루 필요 칼로리 대부분 버터 셈 것 시작 후 고도비만 탈출 4년 60Kg 감량 사실 프랑스 베르나르 르텍시에씨 버터 달걀 고 치즈등 대부분 동물성 식품 구성 식단 적용 후 3개월 몸무게 고혈압 일 가능 동물성 지방 비 동맥경화 고혈압 고지혈증 질환 원인 우리 동물성 지방 풍부 식품 건강 악화 일 분 수 것 사실일까동물성 지방 억울 누명프랑스인들 버터 프랑스 요리 중요 것 세 첫째 버터 둘째 버터 셋째 버터 속담 정도 버터 프랑스 음식 핵심재료 콜레스테롤 신화 버터 프랑스인들 늘어나자(의사들 심혈관계 환자들 버터 치즈 식물성 처방 프랑스 영양학자 피에르 베일 버터 구성 성분 연구 시작 2006년 피에르 웨일 버터 자체 것 버터 소 무엇 먹였느냐 버터 성분 180 사실 발견 이 버터 면죄부 피에르 베일 콜레스테롤 수치 버터 생산 그 발견 기적 버터 방법 단순 옥수수 주성분 곡물사료 대 뿐 1960년대 기점 프랑스 소 풀 건 대 옥수수사료 옥수수 소고 우유 성분 것 옥수수 문제 오메가6 지방산오메가 3 6지방산 우리 몸 세포막 구성 성분 오메가6 지방 축적 오메가3 지방 일 이상적 비율 1 1 원시 조상들 두 지방산 1 1 비율 섭취 현대인 오메가6 오메가3 섭취 비율 20 1 불균형 채 오메가6 지방산 지방세포 증식 염증반응 다양 질환 원인 오메가6 오메가3 지방산 동물 몸 식물만 이 합성 식물 잎 오메가3 지방산 씨앗 오메가6지방산 그 옥수수 알곡 오메가6 구성 비율 예외적 옥수수 오메가6 오메가3 구성 비율 66 1 1970년대 이후 전 세계 여물통(소고 우유 치즈 버터 모이통(닭고 계란 옥수수 알곡 우리 사 우리들 자신 섭생 옥수수 지방 우리들 몸 마음 연구자들 가축 옥수수 섭취 비 심장병 알레르기 질환 증 등 경고 우리 옥수수 습격 것 당신 몸 1 3 옥수수 믿으시겠습니까우리 음식 중 옥수수 기반 것들 취재진 평범 가정 아빠 두사람 머리카락 분석 결과

In [95]:
len(metadata['n_synop'])

1891

In [97]:
len(metadata['epi'])

1891

In [96]:
metadata['n_synop'][0]

'마흔 살 생존 체력 필요 불혹 나이 생애 전환기 신체적 정신적 삶 터닝포인트 40세 정신 일상 치 체력 방전 나이 실감 그동안 체력 중요성 운동 외면 40대 중년들 운동 삶 구원 트레이너 운명적 최소한 시간 맨몸 기초 체력 운동 40대들 저질 체력 극복 프로젝트 생존 체력 약골 40대 저질체력 탈출기 약골 40대 운동 답 코미디 쇼 최강자 전성기 강성범씨 침대 위 아재 40대 중반 체력 중요성 것 몸 생각 육아 입문 5년 차 퇴근 후 두 딸 딸바보 임청묵씨는 날 들 수 만큼 체력 들 자전거 때 번 이상 무리 그 나 올해 마흔 진짜 체력 몸 근력 편 체력 편 40 생존 체력 운동 도전자 임청묵씨 아내 인터뷰 中 운동 것 정신 것 강박관념 압박감 운동 때 걱정 오늘 데 것들 것 거예요 생존 체력 운동 도전자 강성범 인터뷰 中 프리랜서 통역 전문가 대학 강사 서지연씨 공인 체력 아이콘 하루 네 시간 출퇴근 너끈히 소화 커리 우먼 집 소파 일체 그녀 내년 마흔 학창시절 체육시간 김보라씨 모태 저질 체력 그녀 인생 등산 횟수 한 번 정거장 거리 대중교통 이용 귀차니스트 그녀 중년 시간 위 저질 체력 40대 갱생시키 긴급 처방 생존 체력 운동 운동 그 구원해 수 헬스클럽 운동기구 일상 힘 생존 체력 국민체력 테스트 측정 그 현재 체력 등급 외 판정 저질체력 4인방 생존 체력 운동 전문가 아주 트레이너 그곳 운동기구 운동 시작 수 필요 거 하루 15분 시간 하루 15분 시간 그 체력 변화 수 아기 엄마들 아이 수 체력 필요 무게 펀치 것 중요 것 허리 아이 안 수 체력 직장인 회의 야근 수 체력 수험생 공부 책상 위 허리 수 체력 한 것들 거예요 한 것들 실행 수 기본적 밑바닥 체력들 전 생존 체력 생존 체력 운동 전문 아주라 인터뷰 中 생존 체력 운동 스퀏 플랭크 푸시업 버피 네 가지 동작 식스팩 애플 히프 것 일상 최소한 투자(하루 15분 최적 효과(체력 향상 내 것 운동 핵심 헬스클럽 운동 프로그램 생존 체력 운동 저 일상 자연 달 후 그 삶 변'

In [99]:
#이 노트북에선 안씀
#from sklearn.feature_extraction.text import TfidfVectorizer

In [101]:
#n_synoplist = metadata['n_synop']

# CountVectorizer

In [102]:
from sklearn.feature_extraction.text import CountVectorizer

#1개 이하로 나오는 단어 제외
count = CountVectorizer(min_df=2)
#count_matrix = count.fit_transform(metadata)

In [103]:
count_matrix = count.fit_transform(metadata['n_synop'].values.astype('U'))

In [104]:
count_matrix.shape

(1891, 22014)

## 코사인 유사도 점수 계산

In [105]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [106]:
# index는 dataframe의 idx
indices = pd.Series(metadata.index, index=metadata['idx'])

In [107]:
def get_recommendations(idx, cosine_sim2=cosine_sim2):
    #제목과 일치하는 에피소드 인덱스 가지고옴
    idx = indices[idx]
    
    sim_scores = list(enumerate(cosine_sim2[idx]))
    
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    #비슷한 에피소드 상위 10개
    sim_scores = sim_scores[1:11]
    
    # indices
    epi_indices = [i[0] for i in sim_scores]
    
        
    return metadata['epi'].iloc[epi_indices]

### 나중에 epi & recommedation epi, index 한번에 출력 함수 만들기

In [108]:
print('episode : ' + metadata['epi'][236])
get_recommendations(236, cosine_sim2)

episode : 지구의 눈물 북극에서 남극까지 I


235                             지구의 눈물 북극에서 남극까지 II
880                                   신이 내린 묘약 - 눈물
1518                             북극열전 - 제4부 원시북극의 위
1735    북극열전-3부 원시북극의 위기(Entering Virgin Territory)
1519                              북극열전 - 제3부 남획의 그늘
1734             북극열전-4부 남획의 그늘(Predatory activies)
1732              북극열전-6부 영토 분쟁의 서곡(Border Confict)
1517                           북극열전 - 제5부 영토 분쟁의 서곡
1737                             북극열전-1부 위기의 북극을 가다
1736                      북극열전-2부 북극항로, 신해상 실크로드의 꿈
Name: epi, dtype: object

In [123]:
# 결과 이상함................
print('episode : '+ metadata['epi'][887])
get_recommendations(887, cosine_sim2)

episode : 푸른 눈에 내린 신령


362               순례(1)안녕 - 나의 소녀 시절이여(재)
683                  나는 산다 - 김수영, 꿈의 파노라마
813               몸의 유혹 - 그들은 왜 몸에 열광하는가?
734                     하리수 10년, 그녀를 꿈꾸다!
286     Celebrity Biography - 장진영의 마지막 1년
660                 그녀, 뼈를 깎다 - 내 딸의 양악수술
488                   #미투 (Me Too) 나는 말한다
265                 박완서 추모특집 ‘그 겨울은 따뜻했네’
637                            이영애의 만찬 1부
1038                 <4차 인간> 2부 인간은 기계인가?
Name: epi, dtype: object

## 추천 함수?

In [124]:
def recommender(idx):
    print('episode : ' + metadata['epi'][idx])
    
    return get_recommendations(idx, cosine_sim2)
    

In [125]:
recommender(0)

episode : 생존 체력, 약골 40대 저질체력 탈출기


623               ‘간헐적 운동’에서 ‘신체 리모델링’까지
1496        학교체육, 미래를 만나다 2부 운동, 교실을 바꾸다
1042                   생존 - 1부 최후의 생존, 삵
1041                 생존 - 2부 최후의 생존, 너구리
1124                      <사라진 인류> 2부 생존
1040                  생존 - 3부 새, 생존을 말하다
1495    학교체육, 미래를 만나다 3부 운동, 행복한 아이를 만든다
892                    0.2평의 기적, 절하는 사람들
1497       학교체육, 미래를 만나다 1부 체육이 우등생을 만든다
353                플라스틱 지구(2) - 굿바이 플라스틱
Name: epi, dtype: object

In [126]:
recommender(322)

episode : 북에서 온 머구리


879                          심해, 마지막 머구리
807                            연평별곡 (別曲)
866     남측 벼를 심으려 왔습니다 - 평양, 당곡리에서 맺은 결실
841                              말도 아리랑 
781                        영  동  선 (嶺東線)
1439      우랄ㆍ알타이를 가다 - 1부 우랄의 첫 땅, 예레메예보
283                            불가사리와의 전쟁
840                     물위를 떠도는 영혼 바다 집시
844                  SBS스페셜 다큐멘터리 동화 네팔 
729                      어떤 마을에 살고 싶으세요?
Name: epi, dtype: object

In [127]:
# 전부 4차산업혁명 다큐임.
recommender(400)

episode : 돌고돌아 봄 - 불영사에서 나누다


1066    <4차 산업혁명, 교육 패러다임의 대전환> - 2부 칸막이를 없애라
1067      <4차 산업혁명, 교육 패러다임의 대전환> - 1부 대학의 변신
1092           <4차 산업혁명시대 교육대혁명> 2부 평가의 틀을 깨라
1093    <4차 산업혁명시대 교육대혁명> 1부 AI와 인간의 공존은 가능한가
1091          <4차 산업혁명시대 교육대혁명> 3부 대학, 변해야 산다
845                            대한민국 기적의 열쇠 2부
1119                 글로벌 인재전쟁 5부 대한민국, 도약의 조건
1745                                   미래의 학교
1500                  자본주의 5부. 국가는 무엇을 해야 하는가
447                                미래를 바꿀 승부수
Name: epi, dtype: object